# Classification of roads

We're going to look into the success of classifying algorithms on the LiDAR data we have with a couple of different techniques

### The following are the list of classifications defined by IFP
- 0 not yet classified (nothing done yet)
- 1 unclassified (actively marked as nothing)
- 2 ground, sidewalk
- 3,4,5 vegetation, low(gras) medium(shrubbery) high (trees)
- 6 buildings
- 8 street furniture
- 10 street markings
- 11 street, pavement
- 12 bike lanes
- 13 temporary things(bicycles, trashcans)
- 15 cars, trucks

In [1]:
import laspy
import numpy as np
import matplotlib.pyplot as plt     
import seaborn as sns
import open3d as o3d
import pandas as pd

sns.set_theme(style="whitegrid")

In [10]:
def describe_las(las):
    print(f"Point Format: {las.header.point_format}")
    print(f"Number of Points: {las.header.point_count}")
    print("Available Dimensions:", list(las.point_format.dimension_names))
    print("Bounding Box:")
    print(f"  X: {las.header.mins[0]} to {las.header.maxs[0]}")
    print(f"  Y: {las.header.mins[1]} to {las.header.maxs[1]}")
    print(f"  Z: {las.header.mins[2]} to {las.header.maxs[2]}")
    print("Scale:", las.header.scales)
    print("Offset:", las.header.offsets)
    try:
        print("CRS:", las.header.parse_crs())
    except:
        print("CRS: Not defined")

In [6]:
LAZ_FILE = "/Users/jamesmurphy/Downloads/bologna.laz"
las = laspy.read(LAZ_FILE)

In [8]:
max_points = 1_000_000

indices = np.random.choice(len(las.x), max_points, replace=False)
points = np.vstack((las.x[indices], las.y[indices], las.z[indices])).T

In [9]:
las = laspy.read(LAZ_FILE)
x, y, z = las.x, las.y, las.z

In [11]:
describe_las(las)

Point Format: <PointFormat(3, 4 bytes of extra dims)>
Number of Points: 112787346
Available Dimensions: ['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'gps_time', 'red', 'green', 'blue', 'height_division']
Bounding Box:
  X: -319.2701 to -107.22720000000001
  Y: -315.29970000000003 to 26.5196
  Z: -8.614600000000001 to 34.2946
Scale: [0.0001 0.0001 0.0001]
Offset: [0. 0. 0.]
CRS: None


In [13]:
from laspy import read
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Fields of interest
fields = ['X', 'Y', 'Z', 'intensity', 'red', 'green', 'blue', 'classification']

df = pd.DataFrame({field: np.asarray(getattr(las, field)) for field in fields})

# Drop non-ground points (keep classification == 2 or 11 for now)
df = df[df['classification'].isin([2, 11])]  # 2 = ground, 11 = road surface

# Feature engineering placeholders (e.g., flatness, curvature can go here)
# Later...

# Optional: Normalize RGB
df[['red', 'green', 'blue']] /= 65535.0

# Optional: Standard scale numerical features
scaler = StandardScaler()
features = ['Z', 'intensity', 'red', 'green', 'blue']
df[features] = scaler.fit_transform(df[features])


We're dealing with a huge pointcloud here, so let's significantly downsample for the purpose of building the pipeline.

In [ ]:
from sklearn.utils import resample

# Get sidewalk and road separately
df_sidewalk = df[df['classification'] == 2] #sidewalks
df_road = df[df['classification'] == 11] # streets

# Sample a balanced subset
N = 200_000  # 100k each
sidewalk_sample = resample(df_sidewalk, n_samples=N//2, random_state=42)
road_sample = resample(df_road, n_samples=N//2, random_state=42)

df_sampled = pd.concat([sidewalk_sample, road_sample]).sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
df_sampled['label'] = df_sampled['classification'].map({2: 'sidewalk', 11: 'road'})

In [18]:
from sklearn.model_selection import train_test_split

X = df_sampled[features]
y = df_sampled['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[25685  4315]
 [ 4037 25963]]
              precision    recall  f1-score   support

        road       0.86      0.86      0.86     30000
    sidewalk       0.86      0.87      0.86     30000

    accuracy                           0.86     60000
   macro avg       0.86      0.86      0.86     60000
weighted avg       0.86      0.86      0.86     60000



We can also add features to the lidar to improve the model efficacy, such as planes, roughness, etc.

In [23]:
import open3d as o3d

# Convert subset to Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(df_sampled[['X','Y','Z']].values)

# Estimate normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))

# Extract normal z-component to detect flatness (e.g. horizontal surfaces)
normals = np.asarray(pcd.normals)
df_sampled['normal_z'] = normals[:,2]
